## BERT для вопросно-ответных систем

In [1]:
# Если Вы запускаете ноутбук на colab или kaggle,
# выполните следующие строчки, чтобы подгрузить библиотеку dlnlputils:

# !git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
# import sys; sys.path.append('./stepik-dl-nlp')

Скачайте датасет (SQuAD, The Stanford Question Answering Dataset) [отсюда](https://rajpurkar.github.io/SQuAD-explorer/). Для выполенения семинара Вам понадобятся файлы `train-v2.0.json` и `dev-v2.0.json`.

Склонируйте репозиторий https://github.com/huggingface/transformers (воспользуйтесь скриптом `clone_pytorch_transformers.sh`) и положите путь до папки `examples` в переменную `PATH_TO_EXAMPLES`. 

In [2]:
!curl "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json" -o ./train-v2.0.json
!curl "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json" -o ./dev-v2.0.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40.1M  100 40.1M    0     0  19.3M      0  0:00:02  0:00:02 --:--:-- 19.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4268k  100 4268k    0     0  10.0M      0 --:--:-- --:--:-- --:--:-- 10.0M


In [3]:
# !pip uninstall -y transformers

In [4]:
PATH_TO_TRANSFORMERS_REPO = '../transformers/'

In [5]:
import os
os.environ['PATH_TO_TRANSFORMER_REPO'] = PATH_TO_TRANSFORMERS_REPO

In [6]:
#!mkdir ./stepik-dl-nlp/
!curl "https://raw.githubusercontent.com/Samsung-IT-Academy/stepik-dl-nlp/master/clone_pytorch_transformers.sh" \
    -o ./clone_pytorch_transformers.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   164  100   164    0     0    583      0 --:--:-- --:--:-- --:--:--   583


In [7]:
! bash ./clone_pytorch_transformers.sh $PATH_TO_TRANSFORMERS_REPO

Cloning into '../transformers'...
remote: Enumerating objects: 95753, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 95753 (delta 122), reused 136 (delta 66), pack-reused 95500
Receiving objects: 100% (95753/95753), 89.03 MiB | 27.68 MiB/s, done.
Resolving deltas: 100% (70345/70345), done.
remote: Total 0 (delta 0), reused 0 (delta 0), pack-reused 0
From https://github.com/huggingface/transformers
 * branch                74d0bcb6ff692dbaa52da1fdc2b80ece06f5fbe5 -> FETCH_HEAD
HEAD is now at 74d0bcb6f Fix special tokens addition in decoder


In [8]:
!ls ../transformers

CONTRIBUTING.md  README.md  examples	requirements-dev.txt  templates
LICENSE		 docker     hubconf.py	requirements.txt      transformers
MANIFEST.in	 docs	    notebooks	setup.py


In [9]:
import sys

PATH_TO_EXAMPLES = os.path.join(PATH_TO_TRANSFORMERS_REPO, 'examples')

sys.path.insert(1, PATH_TO_EXAMPLES)
sys.path.insert(1, PATH_TO_TRANSFORMERS_REPO)

In [10]:
import torch
import tqdm
import json

from utils_squad import (read_squad_examples, convert_examples_to_features,
                         RawResult, write_predictions,
                         RawResultExtended, write_predictions_extended)

from run_squad import train, load_and_cache_examples

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)

from transformers import (WEIGHTS_NAME, BertConfig, XLNetConfig, XLMConfig,
                          BertForQuestionAnswering, BertTokenizer)

from utils_squad_evaluate import EVAL_OPTS, main as evaluate_on_squad

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

2022-05-19 06:21:59.877714: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

100%|██████████| 440473133/440473133 [00:20<00:00, 21816296.70B/s]


In [12]:
# если Вы не хотите запускать файн-тюнинг, пропустите блок "Дообучение",
# подгрузите веса уже дообученной модели и переходите к блоку "Оценка качества"

# скачайте веса с Google-диска и положите их в папку models
# https://drive.google.com/drive/folders/1-DR30q7MF-gZ51TDx596dAOhgh-uOAPj?usp=sharing

In [13]:
!pip install gdown
!mkdir ./models/

  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14759 sha256=3d0a7c1f965a3bf62d8f4a2cccb41834b88fbe852051f6c7bd58f575d93554f0
  Stored in directory: /root/.cache/pip/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown


In [14]:
# #bert_squad_1epoch.pt
# !gdown 1bqm5KkopcvXPUqJjl_qTN5soy3K1-as6 -O models/

#bert_squad_5epochs.pt
!gdown 17mXqv33_iLcE38qczFxLUQKXpp4acKsD -O models/

Downloading...
From: https://drive.google.com/uc?id=17mXqv33_iLcE38qczFxLUQKXpp4acKsD
To: /kaggle/working/models/bert_squad_5epochs.pt
100%|█████████████████████████████████████████| 438M/438M [00:04<00:00, 106MB/s]


In [15]:
!ls models/ -la

total 427728
drwxr-xr-x 2 root root      4096 May 19 06:23 .
drwxr-xr-x 3 root root      4096 May 19 06:23 ..
-rw-r--r-- 1 root root 437983320 May 19 06:23 bert_squad_5epochs.pt


In [16]:
if torch.cuda.is_available():
    model.cuda()
    model.load_state_dict(torch.load('models/bert_squad_5epochs.pt')) # если у вас есть GPU
else:
    model.load_state_dict(torch.load('models/bert_squad_5epochs.pt', map_location=device)) # если GPU нет

### Дообучение

### Оценка качества работы модели

In [17]:
PATH_TO_DEV_SQUAD = 'dev-v2.0.json'
PATH_TO_SMALL_DEV_SQUAD = 'small_dev-v2.0.json'

with open(PATH_TO_DEV_SQUAD, 'r') as iofile:
    full_sample = json.load(iofile)
    
small_sample = {
    'version': full_sample['version'],
    'data': full_sample['data'][:1]
}

with open(PATH_TO_SMALL_DEV_SQUAD, 'w') as iofile:
    json.dump(small_sample, iofile)

In [18]:
max_seq_length = 384
outside_pos = max_seq_length + 10
doc_stride = 128
max_query_length = 64
max_answer_length = 30

In [19]:
examples = read_squad_examples(
    input_file=PATH_TO_SMALL_DEV_SQUAD,
    is_training=False,
    version_2_with_negative=True)

features = convert_examples_to_features(
    examples=examples,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    doc_stride=doc_stride,
    max_query_length=max_query_length,
    is_training=False,
    cls_token_segment_id=0,
    pad_token_segment_id=0,
    cls_token_at_end=False
)

input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)

example_index = torch.arange(input_ids.size(0), dtype=torch.long)
dataset = TensorDataset(input_ids, input_mask, segment_ids, example_index, cls_index, p_mask)

100%|██████████| 208/208 [00:02<00:00, 77.80it/s]


In [20]:
eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=8)

In [21]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

all_results = []
for idx, batch in enumerate(tqdm.tqdm_notebook(eval_dataloader, desc="Evaluating")):
    model.eval()
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                  }
        inputs['token_type_ids'] = batch[2]
        example_indices = batch[3]
        outputs = model(**inputs)

    for i, example_index in enumerate(example_indices):
        eval_feature = features[example_index.item()]
        unique_id = int(eval_feature.unique_id)
        result = RawResult(unique_id    = unique_id,
                           start_logits = to_list(outputs[0][i]),
                           end_logits   = to_list(outputs[1][i]))
        all_results.append(result)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Evaluating:   0%|          | 0/27 [00:00<?, ?it/s]

In [22]:
all_results[0]

RawResult(unique_id=1000000000, start_logits=[-6.092806339263916, -11.206318855285645, -11.824389457702637, -11.866360664367676, -11.848936080932617, -11.488205909729004, -11.480499267578125, -11.728873252868652, -11.174854278564453, -8.258772850036621, -8.198790550231934, -9.782811164855957, -10.650155067443848, -11.24307918548584, -11.384313583374023, -10.723288536071777, -11.626534461975098, -11.89360523223877, -11.418964385986328, -11.761825561523438, -11.059126853942871, -11.42128849029541, -11.024538040161133, -11.484710693359375, -11.662552833557129, -11.080719947814941, -11.40916919708252, -10.677250862121582, -10.957789421081543, -11.2105712890625, -11.520669937133789, -11.211820602416992, -11.1763334274292, -11.837716102600098, -9.805885314941406, -10.465195655822754, -10.590312957763672, -11.843771934509277, -11.082222938537598, -11.627583503723145, -11.31775951385498, -11.571269035339355, -10.860814094543457, -10.878664016723633, -5.569655895233154, -8.861943244934082, -7.8

In [23]:
n_best_size = 5
do_lower_case = True
output_prediction_file = 'output_1best_file'
output_nbest_file = 'output_nbest_file'
output_na_prob_file = 'output_na_prob_file'
verbose_logging = True
version_2_with_negative = True
null_score_diff_threshold = 0.0

In [24]:
# Генерируем файл с n лучшими ответами `output_nbest_file`
write_predictions(examples, features, all_results, n_best_size,
                    max_answer_length, do_lower_case, output_prediction_file,
                    output_nbest_file, output_na_prob_file, verbose_logging,
                    version_2_with_negative, null_score_diff_threshold)

OrderedDict([('56ddde6b9a695914005b9628', 'France'),
             ('56ddde6b9a695914005b9629', 'the 10th and 11th centuries'),
             ('56ddde6b9a695914005b962a', 'Denmark, Iceland and Norway'),
             ('56ddde6b9a695914005b962b', 'Rollo'),
             ('56ddde6b9a695914005b962c', '10th'),
             ('5ad39d53604f3c001a3fe8d1', ''),
             ('5ad39d53604f3c001a3fe8d2', 'The Normans'),
             ('5ad39d53604f3c001a3fe8d3', 'The Normans'),
             ('5ad39d53604f3c001a3fe8d4', '10th century'),
             ('56dddf4066d3e219004dad5f', 'William the Conqueror'),
             ('56dddf4066d3e219004dad60', 'Richard I of Normandy'),
             ('56dddf4066d3e219004dad61', 'Catholic orthodoxy'),
             ('5ad3a266604f3c001a3fea27',
              'political, cultural and military impact'),
             ('5ad3a266604f3c001a3fea28', ''),
             ('5ad3a266604f3c001a3fea29', ''),
             ('5ad3a266604f3c001a3fea2a', 'Richard I'),
             ('5ad3a266

In [25]:
# Считаем метрики используя официальный SQuAD script
evaluate_options = EVAL_OPTS(data_file=PATH_TO_SMALL_DEV_SQUAD,
                             pred_file=output_prediction_file,
                             na_prob_file=output_na_prob_file)
results = evaluate_on_squad(evaluate_options)

{
  "exact": 65.38461538461539,
  "f1": 68.14217032967032,
  "total": 208,
  "HasAns_exact": 78.125,
  "HasAns_f1": 84.0997023809524,
  "HasAns_total": 96,
  "NoAns_exact": 54.464285714285715,
  "NoAns_f1": 54.464285714285715,
  "NoAns_total": 112,
  "best_exact": 69.71153846153847,
  "best_exact_thresh": -19.841043949127197,
  "best_f1": 72.10851648351647,
  "best_f1_thresh": -10.105830490589142
}


Посмотрим глазами на вопросы и предсказанные БЕРТом ответы:

In [26]:
with open('output_nbest_file', 'r') as iofile:
    predicted_answers = json.load(iofile)

In [27]:
questions = {}
for paragraph in small_sample['data'][0]['paragraphs']:
    for question in paragraph['qas']:
        questions[question['id']] = {
            'question': question['question'],
            'answers': question['answers'],
            'paragraph': paragraph['context']
        }

In [28]:
for q_num, (key, data) in enumerate(predicted_answers.items()):
    gt = '' if len(questions[key]['answers']) == 0 else questions[key]['answers'][0]['text']
    print('Вопрос {0}:'.format(q_num+1), questions[key]['question'])
    print('-----------------------------------')
    print('Ground truth:', gt)
    print('-----------------------------------')   
    print('Ответы, предсказанные БЕРТом:')
    preds = ['{0}) '.format(ans_num + 1) + answer['text'] + \
             ' (уверенность {0:.2f}%)'.format(answer['probability']*100) \
             for ans_num, answer in enumerate(data)]
    print('\n'.join(preds))
#     print('-----------------------------------')   
#     print('Параграф:', questions[key]['paragraph'])
    print('\n\n')

Вопрос 1: In what country is Normandy located?
-----------------------------------
Ground truth: France
-----------------------------------
Ответы, предсказанные БЕРТом:
1) France (уверенность 100.00%)
2) France. (уверенность 0.00%)
3) France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway (уверенность 0.00%)
4) Normandy, a region in France (уверенность 0.00%)
5) France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark (уверенность 0.00%)
6)  (уверенность 0.00%)



Вопрос 2: When were the Normans in Normandy?
-----------------------------------
Ground truth: 10th and 11th centuries
-----------------------------------
Ответы, предсказанные БЕРТом:
1) the 10th and 11th centuries (уверенность 71.27%)
2) 10th and 11th centuries (уверенность 18.23%)
3) in the 10th and 11th centuries (уверенность 10.51%)
4) the 10th (уверенность 0.00%)
5) the 10th and 11th (уверенность 0.00%)

### Запуск обучения через скрипт

In [29]:
# !ls /kaggle/transformers/examples/

In [30]:
# python /kaggle/transformers/examples/run_squad.py \
#   --model_type bert \
#   --model_name_or_path bert-base-cased \
#   --do_train \
#   --do_eval \
#   --do_lower_case \
#   --train_file /kaggle/working/train-v2.0.json \
#   --predict_file /kaggle/working/dev-v2.0.json \
#   --per_gpu_train_batch_size 12 \
#   --learning_rate 3e-5 \
#   --num_train_epochs 2.0 \
#   --max_seq_length 384 \
#   --doc_stride 128 \
#   --output_dir /tmp/debug_squad/